# Producer :

In [ ]:
from confluent_kafka import Consumer

# Configuration du Consumer
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'file_group',
    'auto.offset.reset': 'earliest',
}
consumer = Consumer(conf)

# Souscription au topic
topic = 'files_topic'
consumer.subscribe([topic])

# Fonction pour recevoir et sauvegarder un fichier
def receive_file(output_path):
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue

            # Kafka décompresse automatiquement si compression.type est activée au niveau du producer
            # Sauvegarder les bytes reçus comme un fichier
            with open(output_path, 'wb') as file:
                file.write(msg.value())
                print(f"File received and saved to {output_path}")
                break  # Une fois le fichier reçu, on peut arrêter (si ce n'est pas un flux continu)
    except KeyboardInterrupt:
        print("Consumer interrupted.")
    finally:
        consumer.close()

# Réception du fichier
receive_file("received/motorbikes_received.mp4")


File motorbikes.mp4 sent successfully to topic files_topic


# Consummer :

In [6]:
from confluent_kafka import Consumer

# Configuration du Consumer
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'file_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf)

# Souscription au topic
topic = 'files_topic'
consumer.subscribe([topic])

# Fonction pour recevoir et sauvegarder un fichier
def receive_file(output_path):
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue

            # Sauvegarder les bytes reçus comme un fichier
            with open(output_path, 'wb') as file:
                file.write(msg.value())
                print(f"File received and saved to {output_path}")
    except KeyboardInterrupt:
        print("Consumer interrupted.")
    finally:
        consumer.close()

# Réception de l'image
receive_file("received/motor_bikes_received.mp4")

Consumer interrupted.


In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import time
 
# cap = cv2.VideoCapture(0)  # For Webcam
#cap.set(3, 1280)
#cap.set(4, 720)
cap = cv2.VideoCapture("received/motor_bikes_received.mp4")  # For Video
 
 
model = YOLO("yolov8n.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]
 
prev_frame_time = 0
new_frame_time = 0
 
while True:
    new_frame_time = time.time()
    success, img = cap.read()
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
 
            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)
 
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    print(fps)
 
    cv2.imshow("Image", img)
    cv2.waitKey(1)